### Simulation results

Once your simulation has finished, you probably want to analyse the results with for example
the [ThreediToolbox](https://docs.3di.live/d_qgis_plugin.html) or
[threedigrid](https://threedigrid.readthedocs.io/en/latest/readme.html#). In this notebook
we'll show you how you can download the result files of a 3Di simulation.

Let's first create the client to connect to the threedi-api:

In [ ]:
from getpass import getpass
from pathlib import Path
from threedi_api_client.api import ThreediApi
from threedi_api_client.openapi.api.v3_api import V3Api
from threedi_api_client.files import download_file

In [ ]:
# Provide authentication details
API_HOST = "https://api.3di.live"
PERSONAL_API_KEY = getpass("Personal API token")  # https://management.3di.live/personal_api_keys

config = {
    "THREEDI_API_HOST": API_HOST,
    "THREEDI_API_PERSONAL_API_TOKEN": PERSONAL_API_KEY
}

api_client: V3Api = ThreediApi(config=config)

Now let's select a simulation we're interested in. Make sure it is finished simulating.
We will select the simulation we created in the [previous](./simulation_with_events.ipynb)
notebook.


In [ ]:
USERNAME = input("username for filtering simulations")

simulation_results = api_client.simulations_list(user__username=USERNAME)

if simulation_results.count == 0:
    raise Exception(f"No simulations found for user with username {USERNAME}")

simulation = simulation_results.results[0]

status = api_client.simulations_status_list(simulation.id)

print(simulation)
print(f"status: {status}")
assert status.name == 'finished'

Now that we have our finished simulation lets see which result files are available:

In [ ]:
result_files = api_client.simulations_results_files_list(simulation.id)

for result in result_files.results:
    print(result)

Let's download all the results so we can analyse them locally:

In [ ]:
download_folder = Path(f'results_{simulation.name}')
download_folder.mkdir(exist_ok=True)

for file in result_files.results:
    download = api_client.simulations_results_files_download(
        id=file.id, simulation_pk=simulation.id
    )
    file_path = download_folder / file.filename
    download_file(download.get_url, file_path)
    print(f"Finished downloading {file.filename}")

The ThreediToolbox and threedigrid also require the "gridadmin.h5" file of the simulation.
This is a model specific file so it's under the threedimodels-api. We'll also download this file:

In [ ]:
threedi_model_id = simulation.threedimodel_id
download_url = api_client.threedimodels_gridadmin_download(threedi_model_id)

file_path = download_folder / "gridadmin.h5"
download_file(download.get_url, file_path)

print(f"Finished downloading gridadmin.h5")